In [1]:
# The process will be this:
# 1. Read all dimensions into pandas dfs
# 2. Per record of the fact load, search in the dimension dataframes for the keys to apply
# 3. write the values of these keys into a new dataframe that will be the final df load
# 4. load adjusted dataframe to MSSQL

In [2]:
import pandas as pd

from sqlalchemy import create_engine

from dischargerecord import FactDischarge

In [3]:
# Connect to database
engine = create_engine('mssql://LAPTOP-TH3PDN0I/Group_8_DB?driver=ODBC+Driver+17+for+SQL+Server')

In [4]:
# Load Dimensions
dim_date_df = pd.read_sql_table('DimDate', con=engine).fillna('')
dim_location_df = pd.read_sql_table('DimLocation', con=engine).fillna('')
dim_demographics_df = pd.read_sql_table('DimDemographics', con=engine).fillna('')
dim_payment_df = pd.read_sql_table('DimPayment', con=engine).fillna('')
dim_clinic_class_df = pd.read_sql_table('DimClinicClass', con=engine).fillna('')
dim_apr_class_df = pd.read_sql_table('DimAPRClassification', con=engine).fillna('')
dim_admission_df = pd.read_sql_table('DimAdmission', con=engine).fillna('')
dim_provider_df = pd.read_sql_table('DimProvider', con=engine).fillna('')

In [5]:
# Load facts from csv
fact_discharge_df = pd.read_csv('./sample.csv', 
                                header=0,
                                names=FactDischarge().get_column_names(),
                                dtype=FactDischarge().get_column_types(),
                                converters=FactDischarge().get_column_converters()
                               )
fact_discharge_df = fact_discharge_df.fillna('')
#print(fact_discharge_df)

In [14]:
# Iterate fact dataframe

for fact in fact_discharge_df.itertuples():

    # print(fact)
    
    fact_record = {}

    fact_date_key = dim_date_df.loc[dim_date_df['DischargeYear'] == fact[14]]['DateKey'][0]
    
    fact_provider_key = dim_provider_df[
        (dim_provider_df['AttendingLicenseNo'] == fact[30]) &
        (dim_provider_df['OperatingLicenseNo'] == fact[31]) &
        (dim_provider_df['OtherLicenseNo'] == fact [32])
    ].iloc[0]['ProviderKey']
    
    fact_admission_key = dim_admission_df[
        (dim_admission_df['TypeAdmission'] == fact[12]) &
        (dim_admission_df['PatientDisposition'] == fact[13]) &
        (dim_admission_df['AbortionIndicator'] == fact[34]) &
        (dim_admission_df['EmergencyIndicator'] == fact[35])
    ].iloc[0]['AdmissionKey']
    
    fact_apr_key = dim_apr_class_df[
        (dim_apr_class_df['DrgCode'] == fact[19]) &
        (dim_apr_class_df['MdcCode'] == fact[21]) &
        (dim_apr_class_df['SeverityIllnessCode'] == fact[23]) &
        (dim_apr_class_df['RiskOfMortality'] == fact[25])
    ].iloc[0]['AprKey']
    
    fact_clinic_class_key = dim_clinic_class_df[
        (dim_clinic_class_df['DiagnosisCode'] == fact[15]) &
        (dim_clinic_class_df['ProcedureCode'] == fact[17])
    ].iloc[0]['ClinicClassKey']
    
    fact_payment_key = dim_payment_df[
        (dim_payment_df['PaymentTypology1'] == fact[27]) &
        (dim_payment_df['PaymentTypology2'] == fact[28]) &
        (dim_payment_df['PaymentTypology3'] == fact[29])
    ].iloc[0]['PaymentKey']
    
    fact_demographics_key = dim_demographics_df[
        (dim_demographics_df['AgeGroup'] == fact[6]) &
        (dim_demographics_df['Gender'] == fact[8]) &
        (dim_demographics_df['Race'] == fact[9]) &
        (dim_demographics_df['Ethnicity'] == fact[10])
    ].iloc[0]['DemographicsKey']
    
    fact_location_key = dim_location_df[
        (dim_location_df['HealthServiceArea'] == fact[1]) &
        (dim_location_df['HospitalCounty'] == fact[2]) &
        (dim_location_df['FacilityID'] == fact[4]) &
        (dim_location_df['ZipCode'] == fact[7])
    ].iloc[0]['LocationKey']
    
    fact_record = {
        'ProviderKey': fact_provider_key,
        'DateKey': fact_date_key,
        'AdmissionKey': fact_admission_key,
        'AprKey': fact_apr_key,
        'ClinicClassKey': fact_clinic_class_key,
        'PaymentKey': fact_payment_key,
        'DemographicsKey': fact_demographics_key,
        'LocationKey': fact_location_key,
        'BirthWeight': fact[33],
        'LengthStay': fact[11],
        'TotalCharges': fact[36],
        'TotalCosts': fact[37]
    }
    
    fact_record_df = pd.DataFrame(fact_record, index=[0])
    # print(fact_record_df)
    fact_record_df.to_sql('FactDischarge', if_exists='append', con=engine, index=False)
    

In [ ]:
print(dim_provider_df)

In [ ]:
print(dim_clinic_class_df[
    (dim_clinic_class_df['DiagnosisCode'] == 657) & 
    (dim_clinic_class_df['ProcedureCode'] == 0)
].iloc[0]['ClinicClassKey'])